In [2]:
# imports
import tensorflow as tf
import keras
import pandas as pd

# word to idx
import pickle

# our functions
import functions as f

In [2]:
PATH = {}
PATH["dataset_classification"] = "dataset/classification/"
PATH["dataset_labeling"] = "dataset/seq_labeling/"
PATH["music_reviews_train"] = PATH["dataset_classification"] + "music_reviews_train.json.gz"
PATH["music_reviews_dev"] = PATH["dataset_classification"] + "music_reviews_dev.json.gz"
PATH["music_reviews_test"] = PATH["dataset_classification"] + "music_reviews_test_masked.json.gz"


In [31]:
# Read in google drives
test = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQK7Cyu8BP41ZXqoS74SSHvoBRijNmIwU5p5PrB0RgnDbyWn0DdanuLxaaHTi1J5Fdw71IfsjoYhwJY/pub?gid=0&single=true&output=csv',
                   # Set first column as rownames in data frame
                   index_col=0, error_bad_lines=False
                  )
  # Same result as @TomAugspurger
raw = test.rename(columns={"Annotation (0: negative, 1: positive)": "annotation"})
raw.head(5)

c:\users\music\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,Sentence,annotation,category
index,,,
0,NaN,1.0,NaN
1,This is sad,0.0,NaN
2,I am not happy to see her,0.0,NaN
3,This album is a riot to listen to,1.0,rare words
4,This CD will enchant anyone,1.0,rare words


1 load our model <br>
2 Load the word to idx saved object <br>
3 Find 100 sentencies, or write them. Also with tags ( positive-negative) <br>
4 Try to fuck it up ( swap the meaning, like "good" turn into "not good" and check the result <br>
5 Check accuracy, report hard cases 

In [66]:
test_x = []
test_y = []
test_com = []
for i in range(len(raw)):
    try: 
        len(raw['Sentence'][i])
        test_x.append(raw['Sentence'][i])
        test_y.append(raw['annotation'][i])
        test_com.append(raw['category'][i])
    except:
        continue

Examples from checklist repo

In [91]:
# Sentencies
Train = ['This is a good day', 'what a hell is going on with the sound of the guitar? I never ever heard this shit before, unbeliveable!']
#Train = ['This is bullshit, it sound like an old creppy dog']
Labels = ['positive','negative']
#Labels = ['negative']

# labels to idx
labels = []
for i in Labels:
    if i == "positive":
        labels.append(1)
    else:
        labels.append(0)


# load our model
model = keras.models.load_model('model/2022-3-28-10-35-36')

# load our word to idx
sequencer = pickle.load( open( "model/text_to_seq.p", "rb" ) )
sequencer = pickle.loads(sequencer)

# Transform the example sentencies for idx
Train = sequencer.transform(test_x)

# padding apply
def sequence_pad(sequence, padding='pre', maxlen=50):
    res = tf.keras.preprocessing.sequence.pad_sequences(
        sequence,
        maxlen=maxlen,
        dtype='int32',
        padding=padding,
        truncating='pre',
        value=0.0)
    return res
padding = "post"
maxlen = 50
X_train_p = sequence_pad(Train, padding=padding, maxlen=maxlen)

# prediction
result = model.predict(X_train_p)
#print(result)
print(len(result))

29


## Correct or wrong ?

In [90]:
threshold = 0.5
acc = 0
acc_count = 0
res_final = []
wrongly_classified_text = []
wrongly_classified_original_sentiment = []
for r in result:
    acc_count+=1
    if r<threshold:
        r = 0
        #res_final.append("negative")
        res_final.append(0)
    if r>threshold:
        #res_final.append("positive")
        res_final.append(1)
        
fault_counter = 0
for i in range(len(result)):
    #print("index", i)
    #print("####", res_final[i], "---", hard_sen_sent[i])
    if test_y[i] == res_final[i]:
        #print(" test", test_y[i], "----- res", res_final[i])
        #print("correct")
        continue
    else:
        print("--- wrong, sentence: \n ",test_x[i], "\n---- original sentiment ", test_y[i], "\n result raw: ", result[i])
        fault_counter += 1
        wrongly_classified_text.append(test_x[i])
        wrongly_classified_original_sentiment.append(test_y[i])
print(fault_counter)

--- wrong, sentence: 
  I used to love everything composed by this artist, but these latest songs have just been bad overall 
---- original sentiment  0.0 
 result raw:  [0.51934355]
--- wrong, sentence: 
  I hated this album when I bought it, but it's actually managed to win me over after listening to it a couple of times 
---- original sentiment  1.0 
 result raw:  [0.1203514]
--- wrong, sentence: 
  This album is so ominous that i've been too afraid to go shopping or even turn off the lights for the past 3 days. You absolutely have to listen to this if you are a fan of horror 
---- original sentiment  1.0 
 result raw:  [0.10521677]
--- wrong, sentence: 
  I simply can't understand this album. I hate the choice of instruments, the song genres are all over the place, yet the songs are still repetitve, but i just can't stop listening to this and i don't know why 
---- original sentiment  1.0 
 result raw:  [0.27166283]
--- wrong, sentence: 
  I love the first song however that's it, a